In [134]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [135]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Постановка

Проанализируем данные опроса 4361 женщин из Ботсваны:
`botswana.tsv`

О каждой из них мы знаем:

* сколько детей она родила (признак `ceb`)
* возраст (`age`)
* длительность получения образования (`educ`)
* религиозная принадлежность (`religion`)
* идеальное, по её мнению, количество детей в семье (`idlnchld`)
* была ли она когда-нибудь замужем (`evermarr`)
* возраст первого замужества (`agefm`)
* длительность получения образования мужем (`heduc`)
* знает ли она о методах контрацепции (`knowmeth`)
* использует ли она методы контрацепции (`usemeth`)
* живёт ли она в городе (`urban`)
* есть ли у неё электричество, радио, телевизор и велосипед (`electric`, `radio`, `tv`, `bicycle`)

Давайте научимся оценивать количество детей ceb по остальным признакам.

In [136]:
raw = pd.read_table("botswana.tsv") 
raw.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


Сколько разных значений принимает признак religion?

In [137]:
raw.religion.value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски? 

In [138]:
data = raw

In [139]:
data.dropna(axis=0).shape

(1834, 15)

В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке `agefm` пропуски стоят только там, где `evermarr=0`, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо".

В подобных случаях, когда признак $x1$ на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

* создать новый бинарный признак 
$$x2=\{1, 0; x1= не применимо, иначе;$$
* заменить "не применимо" в $x1$ на произвольную константу ccc, которая среди других значений $x1$ не встречается.

Теперь, когда мы построим регрессию на оба признака и получим модель вида $$y=\beta_0 + \beta_1 x_1 + \beta_2 x_2$$, на тех объектах, где $x1$ было измерено, регрессионное уравнение примет вид $$y=\beta_0 + \beta_1 x$$, а там, где $x1$ было "не применимо", получится $$y=\beta_0 + \beta_1 c + \beta_2$$. Выбор $c$ влияет только на значение и интерпретацию $\beta_2$, но не $\beta_1$.

Давайте используем этот метод для обработки пропусков в `agefm` и `heduc`.

1. Создайте признак `nevermarr`, равный единице там, где в `agefm` пропуски.

In [140]:
data['nevermarr'] = data['agefm'].apply(lambda x: 1 if np.isnan(x) else 0)

2. Удалите признак `evermarr` — в сумме с `nevermarr` он даёт константу, значит, в нашей матрице $X$ будет мультиколлинеарность.

In [141]:
data.drop('evermarr', axis=1, inplace=True)

3. Замените `NaN` в признаке `agefm` на $c_{agefm}=0$.

In [142]:
data['agefm'].fillna(0, inplace=True)

4. У объектов, где `nevermarr = 1`, замените NaN в признаке `heduc` на $c_{heduc_1}=-1$ (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).

In [143]:
data.loc[data['heduc'].isnull() & data['nevermarr']==1, 'heduc'] = -1

Сколько осталось пропущенных значений в признаке heduc?

In [144]:
sum(data['heduc'].isnull())

123

Избавимся от оставшихся пропусков.

Для признаков `idlnchld`, `heduc` и `usemeth` проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (`idlnchld_noans`, `heduc_noans`, `usemeth_noans`), замените пропуски на нехарактерные значения ($c_{idlnchld}=-1$, $c_{heduc_2}=-2$ (значение -1 мы уже использовали), $c_{usemeth}=-1$).

In [145]:
data['idlnchld_noans'] = data['idlnchld'].apply(lambda x: 1 if np.isnan(x) else 0)
data['heduc_noans'] = data['heduc'].apply(lambda x: 1 if np.isnan(x) else 0)
data['usemeth_noans'] = data['usemeth'].apply(lambda x: 1 if np.isnan(x) else 0)

In [146]:
data['idlnchld'].fillna(-1, inplace=True)
data['heduc'].fillna(-2, inplace=True)
data['usemeth'].fillna(-1, inplace=True)

Остались только пропуски в признаках `knowmeth`, `electric`, `radio`, `tv` и `bicycle`. Их очень мало, так что удалите объекты, на которых их значения пропущены.

In [147]:
data.isna().sum()[data.isna().sum()!=0]

knowmeth    7
electric    3
radio       2
tv          2
bicycle     3
dtype: int64

In [148]:
data.dropna(inplace=True)

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик `ceb`).

In [149]:
data.shape[0] * data.shape[1]

78264

5. Постройте регрессию количества детей `ceb` на все имеющиеся признаки методом `smf.ols`.

In [150]:
m1 = smf.ols('ceb ~ + age + educ + religion + idlnchld + knowmeth + usemeth + '\
             'agefm + heduc + urban + electric + radio + tv + bicycle + '\
             'nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted = m1.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Mon, 15 Oct 2018   Prob (F-statistic):               0.00
Time:                        13:35:22   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

Какой получился коэффициент детерминации $R^2$? Округлите до трёх знаков после десятичной точки.

In [151]:
print("{:.3f}".format(fitted.rsquared))

0.644


7. Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

In [152]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [153]:
m1 = smf.ols('ceb ~ + age + educ + religion + idlnchld + knowmeth + usemeth + '\
             'agefm + heduc + urban + electric + radio + tv + bicycle + '\
             'nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)

fitted = m1.fit(cov_type='HC1')

print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Mon, 15 Oct 2018   Prob (F-statistic):               0.00
Time:                        13:35:22   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

8. Удалите из модели незначимые признаки `religion`, `radio` и `tv`. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта. 

In [154]:
m2 = smf.ols('ceb ~ + age + educ + idlnchld + knowmeth + usemeth + '\
             'agefm + heduc + urban + electric + bicycle + '\
             'nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted = m2.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Mon, 15 Oct 2018   Prob (F-statistic):               0.00
Time:                        13:35:22   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.

In [155]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [156]:
m2 = smf.ols('ceb ~ + age + educ + idlnchld + knowmeth + usemeth + '\
             'agefm + heduc + urban + electric + bicycle + '\
             'nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted = m2.fit(cov_type='HC1')
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Mon, 15 Oct 2018   Prob (F-statistic):               0.00
Time:                        13:35:22   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

In [160]:
print("F=%f, p=%.4f, k1=%f" % m1.fit().compare_f_test(m2.fit()))

F=0.919236, p=0.4672, k1=5.000000


9. Признак `usemeth_noans` значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в `usemeth` произвольно выбранным значением $c_{usemeth }=-1$, удалять `usemeth_noans` и `usemeth` можно только вместе.

    Удалите из текущей модели `usemeth_noans` и `usemeth`. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

    Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без `usemeth` и `usemeth_noans`. 

In [161]:
m3 = smf.ols('ceb ~ + age + educ + idlnchld + knowmeth + '\
             'agefm + heduc + urban + electric + bicycle + '\
             'nevermarr + idlnchld_noans + heduc_noans', 
             data=data)
fitted = m3.fit(cov_type='HC1')
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     396.4
Date:                Mon, 15 Oct 2018   Prob (F-statistic):               0.00
Time:                        13:39:58   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.262     -4.

In [166]:
print("F=%f, p=%f, k1=%f" % m2.fit().compare_f_test(m3.fit()))

F=92.890582, p=0.000000, k1=2.000000


In [168]:
m2.fit().compare_f_test(m3.fit())[1]

3.1552009480371243e-40

### Итоговая модель

10. Посмотрите на доверительные интервалы для коэффициентов итоговой модели (не забудьте использовать поправку Уайта, если есть гетероскедастичность ошибки) и выберите правильные выводы.

In [169]:
m2 = smf.ols('ceb ~ + age + educ + idlnchld + knowmeth + usemeth + '\
             'agefm + heduc + urban + electric + bicycle + '\
             'nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted = m2.fit(cov_type='HC1')
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Mon, 15 Oct 2018   Prob (F-statistic):               0.00
Time:                        13:43:51   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

* У женщин, никогда не выходивших замуж, при прочих равных в среднем на 2.3 ребёнка меньше (p<0.001, 95% доверительный интервал для разницы между средними — [-2.6, -1.9])
* С увеличением возраста женщины на 1 год среднее количество детей возрастает на 0.17 (p<0.001, 95% доверительный интервал — [0.16, 0.18])